# 단어에 꼬리표 달기
## 6.1 개체명 인식 모델 훑어보기
### 개체명 인식
- 문장을 토큰화한 뒤 토큰 각각에 인명, 지명, 기관명 등 개체명 태그를 붙여주는 것
- 각 토큰이 어떤 개체명 태그에 속할지 확률을 나타냄
---

## 6.2 개체명 인식 모델 학습하기 (코랩에서 실행)
### .1-2 각종 설정 하기

In [1]:
# 의존성 패키지 설치
# !pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [13]:
# 모델 환경 설정
import torch
from ratsnlp.nlpbook.ner import NERTrainArguments
args = NERTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="ner",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=1,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [14]:
# 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [15]:
# 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters NERTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='named-entity-recognition', downstream_corpus_name='ner', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters NERTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='named-entity-recognition', downstream_corpus_name='ner', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=2, fp16=Fals

### .3 말뭉치 내려받기

In [16]:
nlpbook.download_downstream_dataset(args)

INFO:ratsnlp:cache file(/content/Korpora/ner/train.txt) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/ner/train.txt) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/ner/train.txt) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/ner/val.txt) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/ner/val.txt) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/ner/val.txt) exists, using cache!


### .4 토크나이저 준비

In [17]:
# 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

### .5 데이터 전처리

In [18]:
# 학습 데이터셋 구축
from ratsnlp.nlpbook.ner import NERCorpus, NERDataset
corpus = NERCorpus(args) # 원본문장 + 개체명 태그를 레이블한 문장
train_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_train_BertTokenizer_64_ner_named-entity-recognition [took 1.647 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_train_BertTokenizer_64_ner_named-entity-recognition [took 1.647 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_train_BertTokenizer_64_ner_named-entity-recognition [took 1.647 s]


In [19]:
# 학습 데이터 로더 구축
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [20]:
# 평가용 데이터 로더 구축
val_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_val_BertTokenizer_64_ner_named-entity-recognition [took 0.072 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_val_BertTokenizer_64_ner_named-entity-recognition [took 0.072 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/ner/cached_val_BertTokenizer_64_ner_named-entity-recognition [took 0.072 s]


### .6 모델 불러오기

In [21]:
# 모델 초기화
from transformers import BertForTokenClassification, BertConfig

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForTokenClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: 

### .7 모델 학습시키기

In [22]:
# 태스크 정의
from ratsnlp.nlpbook.ner import NERTask
task = NERTask(model, args)

In [23]:
# 트레이너 정의
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [24]:
# 학습
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-ner exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | mo

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]